In [1]:
import numpy as np
import scipp as sc
import scippneutron as scn

from halt.io import cached_load

In [2]:
rng = np.random.default_rng(8371)

In [3]:
data = cached_load('data/C60_n1e8.h5', 'EventData')
data.coords['spectrum'] = data.coords['spectrum'].astype(sc.dtype.int64)

In [4]:
data

<IPython.core.display.Javascript object>

<scipp.DataArray>
Dimensions: Sizes[spectrum:16384, tof:1, ]
Coordinates:
  position                vector_3_float64              [m]  (spectrum)  [(-0.853647, -0.300000, 157.423139), (-0.849548, -0.300000, 157.420790), ..., (-0.337140, 0.300000, 157.127210), (-0.333041, 0.300000, 157.124861)]
  sample_position         vector_3_float64              [m]  ()  [(-0.692770, 0.000000, 157.100000)]
  source_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 0.000000)]
  spectrum                    int64  [dimensionless]  (spectrum)  [1, 2, ..., 16383, 16384]
  tof                       float64            [µs]  (tof [bin-edge])  [71966.007246, 100000.995220]
Data:
                          DataArrayView  [dimensionless]  (spectrum, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  instrument_name            string  [dimensionless]  ()  ["McStas"]

In [5]:
# Extract events and apply spectrum coord event-wise

spectrum = np.repeat(data.coords['spectrum'].values, data.bins.size().data.values.flatten())
assert len(spectrum) == len(data.events)  # In case not all events are contained in bins.
events = data.events.copy()
del events.coords['pulse_time']
events.coords['spectrum'] = sc.array(dims=['event'], values=spectrum, dtype='int64')
events

<scipp.DataArray>
Dimensions: Sizes[event:98975, ]
Coordinates:
  spectrum                    int64  [dimensionless]  (event)  [39, 39, ..., 16372, 16372]
  tof                       float64            [µs]  (event)  [73267.459740, 79329.848061, ..., 99067.046790, 99507.196418]
Data:
                            float32         [counts]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

In [6]:
# Functions to randomly shuffle 1D DataArrays and Variables

def shuffled_vector(var, indices):
    if len(var.shape) == 0:
        values = var.values
    else:
        values = var.values[indices, :]
    assert var.variances is None
    return sc.vectors(dims=var.dims, values=values, unit=var.unit)
    
def shuffled_variable(var, indices):
    if var.values.ndim > len(var.shape):
        return shuffled_vector(var, indices)
    values = var.values[indices]
    variances = None if var.variances is None else var.variances[indices]
    return sc.array(dims=var.dims, values=values, variances=variances, unit=var.unit)
    
def shuffled(da):
    indices = rng.permutation(len(da))
    out = sc.DataArray(shuffled_variable(da.data, indices),
                       coords={key: shuffled_variable(coord, indices) for key, coord in da.coords.items()},
                       masks={key: shuffled_variable(mask, indices) for key, mask in da.masks.items()},
                       attrs={key: shuffled_variable(attr, indices) for key, attr in da.attrs.items()})
    return out

In [7]:
# Apply the same binning to a DataArray of events as the original loaded data
def bin_like_original(da, original):
    out = sc.bin(da, edges=[original.coords['tof']], groups=[original.coords['spectrum']])
    out.coords['position'] = original.coords['position']
    out.coords['sample_position'] = original.coords['sample_position']
    out.coords['source_position'] = original.coords['source_position']
    return out

In [8]:
wavelength_min = 1.7
wavelength_max = 2.6
wavelength_step = 0.01

In [9]:
distributions = dict()
sample = shuffled(events) 
for n in range(10000, events.shape[0], 10000):
    x = sample['event', :n]
    x = bin_like_original(x, data)
    x = scn.convert(x, 'tof', 'wavelength', scatter=True)
    
    edges = sc.arange('wavelength', wavelength_min, wavelength_max, wavelength_step, unit=sc.units.angstrom)
    # Compute normalised histogram.
    hist = sc.histogram(x, edges)
    hist = hist / sc.sum(hist.data * (edges['wavelength', 1:] - edges['wavelength', :-1]))
    distributions[str(n)] = sc.sum(hist, 'spectrum')

INFO:scippneutron:Ltotal coord or attr not found, trying to compute from L1 and L2
INFO:scippneutron:L2 coord or attr not found, trying to compute from scattered_beam
INFO:scippneutron:scattered_beam coord or attr not found, trying to compute from sample_position and position
INFO:scippneutron:sample_position coord or attr found, using directly
INFO:scippneutron:position coord or attr found, using directly
INFO:scippneutron:L1 coord or attr not found, trying to compute from incident_beam
INFO:scippneutron:incident_beam coord or attr not found, trying to compute from source_position and sample_position
INFO:scippneutron:source_position coord or attr found, using directly
INFO:scippneutron:sample_position coord or attr found, using directly
INFO:scippneutron:Ltotal coord or attr not found, trying to compute from L1 and L2
INFO:scippneutron:L2 coord or attr not found, trying to compute from scattered_beam
INFO:scippneutron:scattered_beam coord or attr not found, trying to compute from sam

In [10]:
sc.plot(distributions)